# NYC Mold violation data processing

### Author: Bartosz Bonczak

In [1]:
import pandas as pd
import numpy as np

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Load Mold violation data

The mold violation data was obtained from NYC Open Data Platform, based on the Department of Housing Preservation and Development (HPD) [Housing Maintenance Code Violations](https://data.cityofnewyork.us/Housing-Development/Housing-Maintenance-Code-Violations/wvxf-dwi5) filtered for rows containing word 'mold' in the 'NOCDescription' column. 

In [2]:
mold_violations = pd.read_csv("../data/big_data/Housing_Maintenance_Code_Violations.csv")
print(mold_violations.shape)
mold_violations.head()

(238922, 41)


,ViolationID,BuildingID,RegistrationID,BoroID,Borough,HouseNumber,LowHouseNumber,HighHouseNumber,StreetName,StreetCode,...,ViolationStatus,RentImpairing,Latitude,Longitude,CommunityBoard,CouncilDistrict,CensusTract,BIN,BBL,NTA
0,10034143,629900,400688,4,QUEENS,77-54,77-44,77-54,AUSTIN STREET,28990,...,Close,N,40.714368,-73.835218,6.0,29.0,76901.0,4079660.0,4.033340e+09,Forest Hills
1,12915832,347379,305459,3,BROOKLYN,2031,2031,2031,OCEAN AVENUE,67530,...,Open,N,40.612377,-73.953858,14.0,48.0,546.0,3182099.0,3.067670e+09,Midwood
2,10000935,81720,208238,2,BRONX,1750,1750,1750,GRAND CONCOURSE,36420,...,Close,N,40.845765,-73.910292,5.0,15.0,22901.0,2007875.0,2.028250e+09,Mount Hope
3,13940473,363196,303800,3,BROOKLYN,425,425,425,ROCKAWAY PARKWAY,73480,...,Close,N,40.657636,-73.916225,17.0,42.0,890.0,3102724.0,3.046940e+09,Rugby-Remsen Village
4,13947557,268557,350988,3,BROOKLYN,378,378,378,EAST 52 STREET,37080,...,Close,N,40.651441,-73.928406,17.0,41.0,862.0,3102896.0,3.046990e+09,Rugby-Remsen Village


# Processing data

In [3]:

# remove duplicates
print(len(mold_violations))
mold_violations.drop_duplicates(inplace=True)
print(len(mold_violations))

238922
238922


In [4]:
# convert InspectionDate to datetime and filter out violations prior 2010
mold_violations['InspectionDate'] = pd.to_datetime(mold_violations['InspectionDate'])
mold_violations = mold_violations[(mold_violations.InspectionDate>'12-31-2009') & (mold_violations.InspectionDate<'01-01-2023')]
mold_violations['CurrentStatusDate'] = pd.to_datetime(mold_violations['CurrentStatusDate'], errors='coerce')
print(len(mold_violations))


214881


In [5]:
# Process BBL and BIN information and drop records with invalid
print(len(mold_violations))
mold_violations['BBl'] = pd.to_numeric(mold_violations.BBL, errors='coerce')
mold_violations['BIN'] = pd.to_numeric(mold_violations.BIN, errors='coerce')
mold_violations.dropna(subset=['BBL', 'BIN'], inplace=True)
mold_violations[['BBL', 'BIN']] = mold_violations[['BBL', 'BIN']].astype(int).astype(str)
mold_violations.BBL = mold_violations.BBL.str.zfill(10)
mold_violations.BIN = mold_violations.BIN.str.zfill(7)
print(len(mold_violations))


214881
214489


In [6]:
cols = ['InspectionDate', 'BBL', 'BIN', 'Latitude','Longitude', 'Class', 'OrderNumber', 'ViolationStatus', 'CurrentStatusDate']

mold_violations = mold_violations[cols]
mold_violations.reset_index(inplace=True, drop=True)

mold_violations['days_to_close'] = (mold_violations.CurrentStatusDate - mold_violations.InspectionDate).dt.days
mold_violations['days_to_close'] = mold_violations.apply(lambda x: x['days_to_close'] if x['ViolationStatus'] == 'Close' else np.nan, axis=1)

In [7]:
mold_violations.sort_values(by='InspectionDate', inplace=True)

In [8]:
mold_violations

,InspectionDate,BBL,BIN,Latitude,Longitude,Class,OrderNumber,ViolationStatus,CurrentStatusDate,days_to_close
72352,2010-01-01,1021080056,1062512,40.833113,-73.941838,B,550,Close,2022-04-06,4478.0
164617,2010-01-02,2048370072,2063384,40.886629,-73.859409,B,550,Close,2014-05-28,1607.0
164620,2010-01-02,4009090055,4020652,40.774522,-73.932765,B,550,Close,2014-09-26,1728.0
164619,2010-01-02,4160480007,4464029,40.596842,-73.798970,B,805,Open,2010-01-05,NaN
164618,2010-01-02,4137120085,4291426,40.657707,-73.749741,B,801,Close,2016-05-26,2336.0
...,...,...,...,...,...,...,...,...,...,...
204195,2022-12-31,3043130047,3097068,40.664901,-73.881306,B,550,Open,2023-01-03,NaN
204216,2022-12-31,1021520045,1063601,40.847544,-73.931690,B,550,Close,2023-04-07,97.0
173282,2022-12-31,3046930056,3102701,40.657238,-73.915811,A,550,Open,2023-04-26,NaN
204217,2022-12-31,2028190005,2007808,40.841006,-73.911299,A,550,Open,2023-01-04,NaN


# Aggregate to BBL level

In [9]:
mold_violations_bbl = mold_violations.groupby(['BBL']).agg(
    {
        'BBL':'count',
        'Latitude':'first',
        'Longitude':'first',
        'InspectionDate':['min',  'max'],
        'Class':'max',
        'ViolationStatus':'last',
        'days_to_close':'mean'

    }
)

mold_violations_bbl.reset_index(inplace=True)
mold_violations_bbl.columns = ['BBL', 'violation_count', 'Latitude','Longitude','first_InspectionDate', 'last_InspectionDate', 'max_Class', 'ViolationStatus', 'avg_days_to_close']
mold_violations_bbl['with_violation'] = 1
mold_violations_bbl

,BBL,violation_count,Latitude,Longitude,first_InspectionDate,last_InspectionDate,max_Class,ViolationStatus,avg_days_to_close,with_violation
0,0000000000,5,40.822745,-73.900336,2022-08-03,2022-10-12,B,Open,147.500000,1
1,1000077501,2,40.703630,-74.010586,2021-04-28,2021-04-28,B,Close,121.000000,1
2,1000157501,2,40.705949,-74.016288,2022-11-07,2022-11-07,B,Close,164.000000,1
3,1000160100,8,40.711244,-74.015925,2010-02-22,2019-10-12,C,Close,903.750000,1
4,1000167519,1,40.716525,-74.014815,2018-10-18,2018-10-18,A,Close,142.000000,1
...,...,...,...,...,...,...,...,...,...,...
33900,5079990012,1,40.513780,-74.247753,2022-06-01,2022-06-01,A,Close,92.000000,1
33901,5080010059,1,40.513274,-74.250740,2010-07-22,2010-07-22,B,Open,NaN,1
33902,5080200185,13,40.516716,-74.240628,2016-06-02,2022-10-05,B,Open,122.000000,1
33903,5080280110,1,40.510679,-74.248867,2020-05-29,2020-05-29,B,Open,NaN,1


In [10]:
mold_violations_bbl.to_csv("../output/bbl_mold_violation.csv", index=False)

# Load 311 mold compaints data

In [11]:
mold_311 = pd.read_csv("../data/big_data/MoldComplaints_311_2010to2022.csv")

mold_311.BBL = mold_311.BBL.astype(str).str.zfill(10)

for c in ['Created Date','Closed Date']:
    mold_311[c] = pd.to_datetime(mold_311[c], errors='coerce').dt.date

mold_311['days_to_comp_close'] = (mold_311['Closed Date'] - mold_311['Created Date']).dt.days

print(mold_311.shape)
mold_311.head()

(312085, 7)


,Unnamed: 0,Unique Key,BBL,Created Date,Descriptor,Closed Date,days_to_comp_close
0,7570,56409867,5063130032,2022-12-31,MOLD,2023-01-12,12
1,7571,56408171,3060240038,2022-12-31,MOLD,2023-01-31,31
2,7572,56410489,3071790058,2022-12-31,MOLD,2023-01-18,18
3,7573,56408325,3050840024,2022-12-31,MOLD,2023-02-13,44
4,7574,56409871,3051120029,2022-12-31,MOLD,2023-01-08,8


In [12]:
mold_311_bbl = mold_311.groupby('BBL').agg(
    {
        'Unique Key':'count',
        'Created Date':['min',  'max'],
        'days_to_comp_close':'mean'
    }
)

mold_311_bbl.reset_index(inplace=True)
mold_311_bbl.columns = ['BBL', 'comp_count', 'first_ComplaintDate', 'last_ComplaintnDate', 'avg_days_to_close_comp']
mold_311_bbl['with_complaint'] = 1
mold_311_bbl

,BBL,comp_count,first_ComplaintDate,last_ComplaintnDate,avg_days_to_close_comp,with_complaint
0,0000000000,16,2010-09-09,2022-12-28,22.500000,1
1,1000077501,1,2017-10-01,2017-10-01,5.000000,1
2,1000077502,1,2012-12-04,2012-12-04,43.000000,1
3,1000130005,3,2015-09-12,2022-12-26,20.000000,1
4,1000150022,1,2013-01-23,2013-01-23,22.000000,1
...,...,...,...,...,...,...
56067,5080370018,1,2012-03-20,2012-03-20,7.000000,1
56068,5080370047,1,2010-01-07,2010-01-07,21.000000,1
56069,5080370053,2,2015-01-07,2015-01-07,2.000000,1
56070,5080390001,1,2016-08-12,2016-08-12,83.000000,1


# Merge Violation with 311 mold complaints data

In [13]:
mold_main = pd.merge(mold_violations_bbl, mold_311_bbl, on='BBL', how='outer')
mold_main['first_mold_issue'] = mold_main[['first_InspectionDate','last_InspectionDate','first_ComplaintDate','last_ComplaintnDate']].min()
mold_main['last_mold_issue'] = mold_main[['first_InspectionDate','last_InspectionDate','first_ComplaintDate','last_ComplaintnDate']].max()
mold_main['mold_issue'] = 1
mold_main

,BBL,violation_count,Latitude,Longitude,first_InspectionDate,last_InspectionDate,max_Class,ViolationStatus,avg_days_to_close,with_violation,comp_count,first_ComplaintDate,last_ComplaintnDate,avg_days_to_close_comp,with_complaint,first_mold_issue,last_mold_issue,mold_issue
0,0000000000,5.0,40.822745,-73.900336,2022-08-03,2022-10-12,B,Open,147.50,1.0,16.0,2010-09-09,2022-12-28,22.500000,1.0,NaT,NaT,1
1,1000077501,2.0,40.703630,-74.010586,2021-04-28,2021-04-28,B,Close,121.00,1.0,1.0,2017-10-01,2017-10-01,5.000000,1.0,NaT,NaT,1
2,1000157501,2.0,40.705949,-74.016288,2022-11-07,2022-11-07,B,Close,164.00,1.0,6.0,2020-03-24,2022-12-16,18.166667,1.0,NaT,NaT,1
3,1000160100,8.0,40.711244,-74.015925,2010-02-22,2019-10-12,C,Close,903.75,1.0,35.0,2010-02-15,2022-06-10,40.857143,1.0,NaT,NaT,1
4,1000167519,1.0,40.716525,-74.014815,2018-10-18,2018-10-18,A,Close,142.00,1.0,2.0,2018-09-25,2020-10-23,17.000000,1.0,NaT,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61175,5080350049,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,1.0,2020-06-12,2020-06-12,54.000000,1.0,NaT,NaT,1
61176,5080370018,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,1.0,2012-03-20,2012-03-20,7.000000,1.0,NaT,NaT,1
61177,5080370047,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,1.0,2010-01-07,2010-01-07,21.000000,1.0,NaT,NaT,1
61178,5080370053,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,2.0,2015-01-07,2015-01-07,2.000000,1.0,NaT,NaT,1


In [14]:
mold_main.to_csv("../output/mold_issues_combined.csv", index=False)